In [277]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy
sns.set(rc={'axes.facecolor':'white', 'figure.facecolor':'white'})

folder = '../data'
merge_file = 'pilot_random1'
plot_folder = '../plots/pilot_random1'

experiments = [
    # {
    #     'pagetimes': 'PageTimes-2021-12-09.csv',
    #     'main_part': 'main_part_pilot_2021-12-09.csv',
    #     'folder': '../data/prepilot'
    # },
    # {
    #     'pagetimes': 'PageTimes-2021-12-13.csv',
    #     'main_part': 'main_part_pilot_2021-12-13.csv',
    #     'folder': '../data/prepilot2'
    # }, 
    # {
    #     'pagetimes': 'PageTimes-2021-12-20.csv',
    #     'main_part': 'main_part_pilot_cont_2021-12-20.csv',
    #     'folder': '../data/prepilot4'
    # }, 
    {
        'pagetimes': 'PageTimes-2022-01-13.csv',
        'main_part': 'main_part_pilot_cont_2022-01-13.csv',
        'folder': '../data/prepilot5',
        'episode_length': 8,
    }, 
    {
        'pagetimes': 'PageTimes-2022-01-27.csv',
        'main_part': 'main_part_pilot_cont_2022-01-27.csv',
        'folder': '../data/pilot1',
        'episode_length': 8,
    }, 
    {
        'pagetimes': 'PageTimes-2022-03-18.csv',
        'main_part': 'main_part_pilot_random_2022-03-18.csv',
        'folder': '../data/prepilotRand1',
        'episode_length': 16,
    }, 
]


session_map = {
    # 'sdvfxj22': 'flextimes', 
    # 'n8op39bm': 'fixedtimes',
    # 'qeklctjv': 'continous_exp',
    # 'a4ly7fp4': 'continous_cheap',
    # 'yi3xaim0': 'trail_rounds',
    'sdvfxj22': 'trail_rounds_2',
    'dbcrara4': 'trail_rounds_2',
    'mqqsqhav': 'trail_rounds_2',
    'yaxdn9bp': 'random_1'
}



exp_pool = {
    'flextimes': 'discrete', 
    'fixedtimes': 'discrete',
    'continous_exp': 'continous',
    'continous_cheap': 'continous',
    'trail_rounds': 'trail_rounds'
}



dfs = []
dfts = []
for exp in experiments:
    df = pd.read_csv(os.path.join(exp['folder'], exp['main_part']))
    dft = pd.read_csv(os.path.join(exp['folder'], exp['pagetimes']))
    print(f"Sessions in folder {exp['folder']} \n{df['session.code'].value_counts()} \n")
    df['experiment_name'] = df['session.code'].map(session_map)
    dft['experiment_name'] = dft['session_code'].map(session_map)
    df['experiment_pool'] = df['experiment_name'].map(exp_pool)
    df['episode_length'] = exp['episode_length']
    dfs.append(df)
    dfts.append(dft)
df = pd.concat(dfs)
dft = pd.concat(dfts)

print(f"Experiments \n{df['experiment_name'].value_counts()} \n")

Sessions in folder ../data/prepilot5 
yi3xaim0    960
1g4f5wxy    160
xl4vfraf    160
Name: session.code, dtype: int64 

Sessions in folder ../data/pilot1 
mqqsqhav    3200
dbcrara4    2720
hmjq6293     160
Name: session.code, dtype: int64 

Sessions in folder ../data/prepilotRand1 
yaxdn9bp    1280
v13x275n     128
lpc387g3     128
e0z791yu      64
bcypr10f      64
Name: session.code, dtype: int64 

Experiments 
trail_rounds_2    5920
random_1          1280
Name: experiment_name, dtype: int64 



In [278]:
df['subsession.round_number'].value_counts()

1     564
2     564
3     564
4     564
5     564
6     564
7     564
8     564
9     564
10    564
11    564
12    564
13    564
14    564
15    564
16    564
Name: subsession.round_number, dtype: int64

In [279]:
where = (
    (df['player.role'] != 'manager') &
    (~df['player.role'].isnull()) 
)

df = df[where]

assert df.groupby(['session.code', 'subsession.round_number', 'participant.code'])['participant.label'].count().max() == 1

In [280]:
df[(df['participant._current_page_name'] != 'FinalScreen')]['participant.code'].value_counts()

i9836y9t    16
887qqdaa    16
wg8x68y9    16
8eemo62g    16
7cxtfooj    16
u54y917r    16
b3vbtcpq    16
3wtegng4    16
iiphaol9    16
pbigtcdg    16
w66fqk33    16
43htuytx    16
tydz2zl6    16
wbaq4gyo    16
tpirimw7    16
wd28mvpk    16
3u4ncuct    16
16lxok93    16
9flsyy5l    16
stg9swa9    16
mnwvuwhe    16
mqjvdqyr    16
cs8vkk8v    16
gcknvbz9    16
59k1azdw    16
7ob1jjat    16
9568g1ze    16
5teut236    16
mqv6atek    16
a4lk7dhq    16
dimn40h6    16
v6d1zdj9    16
w758j3io    16
Name: participant.code, dtype: int64

In [281]:
df[(df['participant._current_page_name'] != 'FinalScreen')]['player.contribution'].value_counts(dropna=False)

NaN     368
0.0     102
20.0     31
16.0      7
17.0      3
5.0       3
14.0      3
7.0       3
15.0      3
11.0      2
4.0       1
18.0      1
3.0       1
Name: player.contribution, dtype: int64

In [282]:

max_player = 4

def get_punishment(row):
    member_idx = row['player.role'][-1]
    return row[f"group.punishment{member_idx}"]

rename = {
    'session.code': 'session', 
    'group.id_in_subsession': 'group_id', 
    'subsession.round_number': 'round_number',
    'participant.code': 'participant_code',
    'player.contribution': 'contribution',
    'player.no_input': 'player_no_input',
    'player.contribution': 'contribution',
    'player.punishment_s': 'punishment_rule_s',
    'player.punishment_b': 'punishment_rule_b',
    'player.punishment_c': 'punishment_rule_c',
    'player.punishment': 'punishment',
    'group.no_input_manager': 'manager_no_input',
    'group.total_contribution': 'total_contribution',
    'group.common_good': 'common_good',
    'group.total_punishment': 'total_punishment',
    'episode': 'episode',
    'experiment_name': 'experiment_name',
    'player.role': 'player_role',
    'player.payoff': 'payoff',
    'experiment_pool': 'experiment_pool',
    'episode_length': 'episode_length',
}

df['episode'] = (df['subsession.round_number'] - 1) // df['episode_length'] + 1


df['player.punishment'] = df.apply(get_punishment, axis=1)


df = df[[k for k in rename.keys() if k in df.columns]]
df = df.rename(columns=rename)

df['round_number'] = (df['round_number']-1) % df['episode_length'] 
# dft = dft[dft['page_name'] == 'Results']
# merge_on = ['experiment_name', 'round_number', 'participant_code']

df['global_group_id'] = df['session'] + ' #' + df['group_id'].astype('str')

df['player_id'] = df['player_role'].str.strip().str[-1].astype(int) - 1


# df = df.merge(dft[merge_on + ['epoch_time_completed']], on=merge_on, how='left')


# df['group_started'] = df.groupby(['group_id', 'session'])['epoch_time_completed'].transform('min')
# df['global_group_id'] = df['group_started'].rank(method='dense').astype(int)

# df['time_passed'] = df['epoch_time_completed'] - df.groupby('participant_code')['epoch_time_completed'].transform('min')
# df['group_duration'] = df.groupby('global_group_id')['time_passed'].transform('max')
# df['group_duration_min'] = df['group_duration'] / 60


In [283]:
# removing unknown sessions (i.e. test sessions)
w_no_exp = df['experiment_name'].isnull()

print(f"{w_no_exp.sum()} removed rows.")
print(f"Removed Sessions \n {df.loc[w_no_exp, 'session'].value_counts()}")

df = df[~w_no_exp]

848 removed rows.
Removed Sessions 
 yi3xaim0    400
1g4f5wxy     64
xl4vfraf     64
hmjq6293     64
e0z791yu     64
bcypr10f     64
v13x275n     64
lpc387g3     64
Name: session, dtype: int64


In [284]:
df['round_number'].value_counts()

0     382
1     382
2     382
3     382
4     382
5     382
6     382
7     382
8      38
9      38
10     38
11     38
12     38
13     38
14     38
15     38
Name: round_number, dtype: int64

In [285]:
df[df['contribution'].isnull()]['player_no_input'].value_counts()

0    76
Name: player_no_input, dtype: int64

In [286]:
# removing out participants with no input in any round

w = df.groupby(['global_group_id', 'episode', 'participant_code'])['player_no_input'].transform('all')
print(f"{w.sum()} removed rows.")
df = df[~w]

0 removed rows.


In [287]:
df.groupby(['global_group_id', 'episode', 'participant_code'])

In [288]:
groupby = ['global_group_id', 'round_number', 'episode']

check = (df['punishment'].isnull() == df['contribution'].isnull())
assert check.all(), 'If punishment is null, contribution is null as well.'
n_p_null = df['punishment'].isnull().sum()
print(f"""
There are {n_p_null} rows with null punishments. These are likely from aborted games. We will remove these rows.
""")
df = df[~df['punishment'].isnull()]

if 'manager_no_input' in df.columns:
    check = (df['manager_no_input'] == (df['punishment'] == 0))
    assert check.any(), 'Inconsistency!'

check = (df['player_no_input'] == (df['contribution'] == 0))
assert check.any(), 'Inconsistency!'

check = df.groupby(groupby)['total_contribution'].transform('first') == df.groupby(groupby)['contribution'].transform('sum')
assert check.all(), 'Sum of total contributions does not match group contribution.'


There are 76 rows with null punishments. These are likely from aborted games. We will remove these rows.



In [289]:
# for debugging

df[~check].sort_values(['session','group_id', 'round_number'])

,session,group_id,round_number,participant_code,contribution,player_no_input,punishment_rule_s,punishment_rule_b,punishment_rule_c,punishment,...,common_good,total_punishment,episode,experiment_name,player_role,payoff,experiment_pool,episode_length,global_group_id,player_id


In [290]:
# we might have different number of rows per round, due to some participants dropping out
check = df.groupby(groupby)['total_contribution'].count()
check.value_counts()

4    821
Name: total_contribution, dtype: int64

In [291]:
# we have different number of round per episode, due to some episodes failing early
check = df.groupby(groupby[:-1])['total_contribution'].count()
check.value_counts()

8    337
4    147
Name: total_contribution, dtype: int64

In [292]:
df['episode_id'] = (df['global_group_id'] + df['episode'].astype(str)).rank(method='dense').astype(int) - 1

/var/folders/nv/p5tx0nn545v9h6_w1wtm8cg00000gn/T/ipykernel_50865/2202981932.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['episode_id'] = (df['global_group_id'] + df['episode'].astype(str)).rank(method='dense').astype(int) - 1


In [293]:
df.to_csv(os.path.join(folder, f'{merge_file}_player_round.csv'), index=False)

In [294]:
selected_columns = ['session', 'global_group_id','episode', 'episode_id', 'experiment_name', 'round_number', 'participant_code', 'player_no_input', 'player_id', 'contribution', 'punishment', 'payoff', 'common_good']

df[selected_columns].to_csv(os.path.join(folder, f'{merge_file}_player_round_slim.csv'), index=False)